# Exploration of the ERA-int monthly data collection

[Database Exploration](#db)

[Indexes](#indexes)

[SLP plot](#slpplot)

In [41]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pymongo
from pprint import pprint
from datetime import datetime, timedelta, date
import pandas as pd

## Database exploration <a id='db'></a>

In [2]:
mongo_host_local = 'mongodb://localhost:27017/'
mg = pymongo.MongoClient(mongo_host_local)

In [3]:
pprint(mg.database_names())

['ECMWF', 'local']


In [4]:
db = mg.ECMWF
db.collection_names()

['ERAINT_grid', 'ERAINT_monthly', 'system.indexes']

In [12]:
con_grid = db.ERAINT_grid
fo = con_grid.find_one()

In [13]:
pprint(fo)

{'_id': ObjectId('5a088e938cb6b80d9b4c7f93'),
 'id_grid': 1,
 'loc': {'coordinates': [-180.0, 90.0], 'type': 'Point'}}


In [14]:
con_grid.count()

115680

In [16]:
con_data = db.ERAINT_monthly
fo = con_data.find_one()
pprint(fo)

{'_id': ObjectId('5a089e0f8cb6b81914062592'),
 'al': 0.07,
 'ci': 1.0,
 'd2m': 235.27,
 'date': datetime.datetime(1979, 2, 1, 0, 0),
 'hcc': 0.2,
 'id_grid': 1,
 'istl1': 238.26,
 'lcc': 0.58,
 'mcc': 0.26,
 'msl': 101948.67,
 'si10': 6.1,
 'skt': 238.46,
 'sp': 101948.57,
 'sst': 271.46,
 'stl1': 238.47,
 't2m': 238.56,
 'u10': 1.18,
 'v10': -0.01,
 'year': 1979}


In [17]:
this_day = datetime(1995,1 ,1)
for doc in con_data.find({'date': this_day}).limit(2):
    pprint(doc)

{'_id': ObjectId('5a08a9bf8cb6b81914771dd2'),
 'al': 0.07,
 'ci': 1.0,
 'd2m': 239.33,
 'date': datetime.datetime(1995, 1, 1, 0, 0),
 'hcc': 0.29,
 'id_grid': 1,
 'istl1': 242.21,
 'lcc': 0.52,
 'mcc': 0.19,
 'msl': 102341.94,
 'si10': 4.96,
 'skt': 242.22,
 'sp': 102341.87,
 'sst': 271.46,
 'stl1': 242.24,
 't2m': 242.46,
 'u10': -0.5,
 'v10': 2.28,
 'year': 1995}
{'_id': ObjectId('5a08a9bf8cb6b81914771dd3'),
 'al': 0.07,
 'ci': 1.0,
 'd2m': 239.33,
 'date': datetime.datetime(1995, 1, 1, 0, 0),
 'hcc': 0.29,
 'id_grid': 2,
 'istl1': 242.21,
 'lcc': 0.52,
 'mcc': 0.19,
 'msl': 102341.94,
 'si10': 4.96,
 'skt': 242.22,
 'sp': 102341.87,
 'sst': 271.46,
 'stl1': 242.24,
 't2m': 242.46,
 'u10': -0.46,
 'v10': 2.27,
 'year': 1995}


In [18]:
# How many documents are in the daily data collection
con_data.count()

53675520

In [19]:
alldays = con_data.distinct(key='date')
alldays

[datetime.datetime(2017, 8, 1, 0, 0),
 datetime.datetime(2017, 7, 1, 0, 0),
 datetime.datetime(2017, 6, 1, 0, 0),
 datetime.datetime(2017, 5, 1, 0, 0),
 datetime.datetime(2017, 4, 1, 0, 0),
 datetime.datetime(2017, 3, 1, 0, 0),
 datetime.datetime(2017, 2, 1, 0, 0),
 datetime.datetime(2017, 1, 1, 0, 0),
 datetime.datetime(2016, 12, 1, 0, 0),
 datetime.datetime(2016, 11, 1, 0, 0),
 datetime.datetime(2016, 10, 1, 0, 0),
 datetime.datetime(2016, 9, 1, 0, 0),
 datetime.datetime(2016, 8, 1, 0, 0),
 datetime.datetime(2016, 7, 1, 0, 0),
 datetime.datetime(2016, 6, 1, 0, 0),
 datetime.datetime(2016, 5, 1, 0, 0),
 datetime.datetime(2016, 4, 1, 0, 0),
 datetime.datetime(2016, 3, 1, 0, 0),
 datetime.datetime(2016, 2, 1, 0, 0),
 datetime.datetime(2016, 1, 1, 0, 0),
 datetime.datetime(2015, 12, 1, 0, 0),
 datetime.datetime(2015, 11, 1, 0, 0),
 datetime.datetime(2015, 10, 1, 0, 0),
 datetime.datetime(2015, 9, 1, 0, 0),
 datetime.datetime(2015, 8, 1, 0, 0),
 datetime.datetime(2015, 7, 1, 0, 0),
 datet

## Data collection indexes <a id='indexes'></a>

In [36]:
ind = con_grid.index_information()
pprint(ind)

{'_id_': {'key': [('_id', 1)], 'ns': 'ECMWF.ERAINT_grid', 'v': 1},
 'loc_2dsphere_id_grid_1': {'2dsphereIndexVersion': 2,
                            'key': [('loc', '2dsphere'), ('id_grid', 1)],
                            'ns': 'ECMWF.ERAINT_grid',
                            'v': 1}}


In [20]:
con_data.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'ECMWF.ERAINT_monthly', 'v': 1},
 'date_-1': {'key': [('date', -1)], 'ns': 'ECMWF.ERAINT_monthly', 'v': 1},
 'id_grid_1_date_-1': {'key': [('id_grid', 1), ('date', -1)],
  'ns': 'ECMWF.ERAINT_monthly',
  'v': 1},
 'year_1_id_grid_1': {'key': [('year', 1), ('id_grid', 1)],
  'ns': 'ECMWF.ERAINT_monthly',
  'v': 1}}

# Plot SLP <a id='slpplot'></a>

In [21]:
con_grid.find_one()

{'_id': ObjectId('5a088e938cb6b80d9b4c7f93'),
 'id_grid': 1,
 'loc': {'coordinates': [-180.0, 90.0], 'type': 'Point'}}

- The domain north of 20°N is chosen for EOF decomposition for Z70 hPa and SIC to focus on the extra-tropical variability 
- The area north of 20°S is chosen for SST to also include key regions of tropical SST variability

In [80]:
slp_poly = [[180,20], [180,90],[-180,90], [-180,20],[180,20]] #[[-180,20],[-180,0],[180,0],[-180,0],[-180,20]]

In [81]:
this_polygon = slp_poly
geo_qry = {"loc": 
           {"$geoWithin": {
               "$geometry": {
                   "type": "Polygon","coordinates": [this_polygon]}}}}
con_grid.count(filter = geo_qry)

573

In [73]:
res = con_grid.find(filter = geo_qry, projection = {"_id":0, "id_grid": 1, "loc": 1})
grid_df = pd.DataFrame(list(res))
grid_ids = grid_df.id_grid.values
grid_ids[:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [70]:
# Query SLP in the data collection for a given day for the grid_ids
res = con_data.find(filter = {
    "date": datetime(1988, 12, 1, 0, 0),
    "id_grid": {"$in": grid_ids.tolist()}},
                       projection = {"_id":0, "id_grid": 1, "msl": 1})
slp_df = pd.DataFrame(list(res))
slp_df.head()

,id_grid,msl
0,1,99672.69
1,2,99672.69
2,3,99672.69
3,4,99672.69
4,5,99672.69


In [75]:
grid_df.head()

,id_grid,loc
0,1,"{'type': 'Point', 'coordinates': [-180.0, 90.0]}"
1,2,"{'type': 'Point', 'coordinates': [-179.25, 90.0]}"
2,3,"{'type': 'Point', 'coordinates': [-178.5, 90.0]}"
3,4,"{'type': 'Point', 'coordinates': [-177.75, 90.0]}"
4,5,"{'type': 'Point', 'coordinates': [-177.0, 90.0]}"


In [77]:
# Merge slp_df with grid_df:
slp_dat = pd.merge(left=slp_df, right=grid_df, on = "id_grid")
slp_dat.head()

,id_grid,msl,loc
0,1,99672.69,"{'type': 'Point', 'coordinates': [-180.0, 90.0]}"
1,2,99672.69,"{'type': 'Point', 'coordinates': [-179.25, 90.0]}"
2,3,99672.69,"{'type': 'Point', 'coordinates': [-178.5, 90.0]}"
3,4,99672.69,"{'type': 'Point', 'coordinates': [-177.75, 90.0]}"
4,5,99672.69,"{'type': 'Point', 'coordinates': [-177.0, 90.0]}"


In [71]:
con_grid.find_one()

{'_id': ObjectId('5a088e938cb6b80d9b4c7f93'),
 'id_grid': 1,
 'loc': {'coordinates': [-180.0, 90.0], 'type': 'Point'}}

In [79]:
import GDAL as gdl

ModuleNotFoundError: No module named 'GDAL'